In [ ]:
from datetime import date
from sqlalchemy import create_engine, text
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import matplotlib.pyplot as plt 
import platform
import os
import requests
import re
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# MariaDB에 연결하기
db_id = ''
db_pw = ''
db_ip = ''
db_port = 0
db_name = ''

engine = create_engine(f"mysql+pymysql://{db_id}:{db_pw}@{db_ip}:{db_port}/{db_name}?charset=utf8mb4")
conn = engine.connect()

In [2]:
ticke_df = pd.read_csv('./stockcode1.2.csv', dtype=object)
tk = ticke_df['종목코드']

,종목코드,종목명
0,060310,3S
1,095570,AJ네트웍스
2,006840,AK홀딩스
3,054620,APS
4,265520,AP시스템
...,...,...
1514,189980,흥국에프엔비
1515,000540,흥국화재
1516,003280,흥아해운
1517,037440,희림


In [4]:
# stlist라는 빈 리스트를 생성합니다.
stlist = []
# tk 리스트를 순회하면서 각 종목코드에 대해 다음 작업을 수행합니다.
# 해당 코드를 실행하면, 주어진 종목코드 리스트의 각 종목에 대한 발행주식수, 유동주식수, 유동비율 정보를 크롤링하여 stlist에 저장합니다.
for ticker  in tk:
    try:
# columns 리스트에 필요한 컬럼명을 저장합니다.
        columns = ['종목코드', '발행주식수', '유동주식수','유동비율']
# stock_df라는 빈 DataFrame을 생성합니다.
        stock_df = pd.DataFrame(columns=columns)
        stock_df
# 크롤링할 URL을 지정합니다.
        url = f'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={ticker}'
#네이버는 헤더를 꼭 줘야 크롤링 가능
# requests.get() 함수를 사용하여 URL에서 HTML 페이지를 가져옵니다.
        req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
# BeautifulSoup 모듈을 사용하여 HTML 페이지를 파싱합니다.
        html = BeautifulSoup(req.text, "lxml")
# 파싱한 HTML에서 필요한 정보가 있는 테이블을 찾아 DataFrame으로 변환합니다.
        ctb11 = html.find('table', class_='gHead', id='cTB11')
        ctb13 = html.find('table', class_='gHead01 all-width', id='cTB13')
# 변환한 DataFrame에서 필요한 정보를 추출하여 stock_df에 저장합니다.
        ctb11_text = str(ctb11)
        ticker_df = pd.read_html(ctb11_text)[0]
        ctb13_text = str(ctb13)
        tickers_df = pd.read_html(ctb13_text)[0]
        tickers_df.fillna(0,inplace=True)
        ticker_df.set_index(0, inplace=True)
        shares_issued = ticker_df.loc['발행주식수/유동비율']
        num_shares_issued = shares_issued[1].split('/')[0]
        num_shares_issued = re.sub('[^0-9]','', num_shares_issued)
        num_shares_issued = int(num_shares_issued)
        free_float_rate =100 - tickers_df['보유지분(%)'].sum()
        free_issued_rate = round((num_shares_issued * free_float_rate / 100))
        pd.set_option('display.float_format', '{:.2f}'.format)
        stock_df.loc[len(stock_df)] = [ticker, num_shares_issued, free_issued_rate, free_float_rate]
# stock_df를 stlist에 추가합니다.
        stlist.append(stock_df)   
# 종목코드를 출력합니다.
        print(ticker)
        
    except Exception as e:
# 예외 처리를 위해 try-except 구문을 사용합니다. 크롤링 중 오류가 발생하면 오류 메시지를 출력합니다.
        print(f"{ticker} 크롤링 중 오류 발생: {e}")

060310
095570
006840
054620
265520
211270
027410
126600
001460
013720
083790
079160
311690
011150
051500
058820
023460
056730
000480
000590
012030
016610
025440
001530
001880
068790
007340
004840
241520
155660
069730
180400
017940
037370
007700
214270
114090
900290
083450
001250
297890
078150
012630
039570
089470
294870
322000
097230
014790
003580
278650
024850
047920
115450
046210
060980
036640
082740
035000
234080
067290
001060
096760
318000
021320
344820
036670
009070
009440
119650
092220
151860
046440
016380
035600
001390
033180
060720
015590
001940
058400
025000
092230
044450
058850
058860
122450
052900
376190
061970
093050
037560
000680
060370
229640
108670
383800
086960
160550
053290
181710
060250
104200
034310
008260
004250
178920
024940
218410
327260
019550
950110
034120
151910
036120
099220
036540
255220
005090
016250
001380
004060
002360
009160
006120
210980
068400 크롤링 중 오류 발생: Missing optional dependency 'html5lib'.  Use pip or conda to install html5lib.
001510
048550
06344

In [5]:
# 앞에서 데이터프레임 합치기
result = pd.concat(stlist, axis=0)
re = pd.DataFrame(result)

In [ ]:
# fdr.StockListing('KRX') 함수를 사용하여 KRX에서 상장된 주식 리스트를 가져옵니다.
df_krx = fdr.StockListing('KRX')
# isin() 함수를 사용하여 tk 리스트에 포함된 종목코드를 가진 행만 선택합니다.
selected_stocks = df_krx[df_krx.iloc[:,0].isin(tk)]
# 선택한 행에서 'Marcap' 열의 값을 리스트로 변환하여 columns 변수에 저장합니다.
columns = selected_stocks['Marcap'].values.tolist()

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,73100,1,100,0.14,73800,73900,72700,15092478,1103375423000,436391104405000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,156500,1,7500,5.03,154500,156500,152600,5925908,918226745108,113932370122500,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,408500,1,1500,0.37,410000,415500,405000,177106,72706138000,95589000000000,234000000,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,820000,2,-3000,-0.36,825000,832000,818000,39665,32643973500,58362680000000,71174000,STK
4,005935,KR7005931001,삼성전자우,KOSPI,,63200,1,100,0.16,63000,63200,62700,994430,62642890200,52006439440000,822886700,STK


In [ ]:
# 시작 시퀀스 처리코드
result = conn.execute(text("SELECT COALESCE(MAX(s.shseq), 0) + 1 AS next_shseq FROM SHAREHOLD s"))
next_shseq = result.scalar()

In [ ]:
#현재날짜 처리코드
today = date.today()
# 각각시퀀스처리코드
new_seq = []
for idx, row in re.iterrows():
    next_shseq += 1
    new_seq.append(next_shseq)


2024-02-22


In [ ]:
# 데이터프레임을 DB 및 CSV로 저장코드
re.insert(0,'shseq', new_seq)
re['유동비율'] = re['유동비율'].round().astype(int)
re['종목코드']= re['종목코드'].apply(lambda x: 'A'+ str(x))
re['marketcap'] = columns
re.insert(2,'shrdate',today)
re.columns = ['shseq','shrtk','shrdate','listedstock','floatingstock','floatingratio','marketcap']
re.to_csv('resultsa.csv', index=False)
re.to_sql('SHAREHOLD', conn, if_exists='append', index=False)
conn.commit()
conn.close()